In [1]:
import pandas as pd
from pywikidata import Entity
from tqdm.auto import tqdm
import ujson
import itertools
import random
import datasets
from kbqa.entity_linkink import build_mgenre_pipeline, EntitiesSelection
from kbqa.caches.ner_to_sentence_insertion import NerToSentenceInsertion

tqdm.pandas()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-06 14:53:59.996607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 14:54:00.132818: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-06 14:54:00.165111: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register 

In [2]:
import requests
from joblib import Memory

memory = Memory('/tmp/cache', verbose=0)


@memory.cache
def get_wd_search_results(
    search_string: str,
    max_results: int = 500,
    language: str = 'en',
    mediawiki_api_url: str = "https://www.wikidata.org/w/api.php",
    user_agent: str = None,
) -> list:
    params = {
        'action': 'wbsearchentities',
        'language': language,
        'search': search_string,
        'format': 'json',
        'limit': 50
    }

    user_agent = "pywikidata" if user_agent is None else user_agent
    headers = {
        'User-Agent': user_agent
    }

    cont_count = 1
    results = []
    while cont_count > 0:
        params.update({'continue': 0 if cont_count == 1 else cont_count})

        reply = requests.get(mediawiki_api_url, params=params, headers=headers)
        reply.raise_for_status()
        search_results = reply.json()

        if search_results['success'] != 1:
            raise Exception('WD search failed')
        else:
            for i in search_results['search']:
                results.append(i['id'])

        if 'search-continue' not in search_results:
            cont_count = 0
        else:
            cont_count = search_results['search-continue']

        if cont_count > max_results:
            break

    return results

In [3]:
def prepare_data(data_df, results_df, wd_search_results_top_k: int = 1, mgenre=None, ner=None, entities_selection=None):
    answers_cols = [c for c in results_df.columns if 'answer_' in c]

    results_df['answers_ids'] = results_df[answers_cols].progress_apply(
        lambda row: [
            get_wd_search_results(label, 5, language='en')[:wd_search_results_top_k]
            for label in row.unique()[:5]
        ],
        axis=1
    ).apply(lambda list_of_list_of_answers: list(itertools.chain(*list_of_list_of_answers)))

    df = results_df.merge(data_df, on='question')

    for _, row in tqdm(df.iterrows(), total=df.index.size):
        if len(row['answerEntity']) == 0:
            continue

        golden_true_entity = [Entity(e['name']) for e in row['answerEntity']]

        if mgenre is None or ner is None or entities_selection is None:
            question_entity = [
                Entity(e['name'])
                for e in row['questionEntity']
                if e['entityType'] == 'entity' and e['name'] not in [None, 'None', ''] and e['name'][0] == 'Q'
            ]
            additional_candidates = []
            for qe in question_entity:
                for _,e in qe.forward_one_hop_neighbours:
                    if e not in golden_true_entity:
                        additional_candidates.append(e.idx)
            random.shuffle(additional_candidates)
            additional_candidates = additional_candidates[:5]
        else:
            question_with_ner, entities_list = ner.entity_labeling(row['question'], True)
            mgenre_results = mgenre(question_with_ner)
            selected_entities = entities_selection(entities_list, mgenre_results)

            questionEntity = list(itertools.chain(*[
                get_wd_search_results(l, 1, language='en')[:1]
                for l in selected_entities
            ]))
            additional_candidates = []

        candidates_ids = set(additional_candidates + row['answers_ids'] + [e.idx for e in golden_true_entity])
        
        for candidate_id in candidates_ids:
            candidate_entity = Entity(candidate_id)
            yield {
                'id': row['id'],
                'question': row['question'],
                'answerEntity': [candidate_entity.idx],
                'questionEntity': [e.idx for e in question_entity],
                'groundTruthAnswerEntity': [e.idx for e in golden_true_entity],
                'complexityType': row['complexityType'],
            }


In [4]:
ds = datasets.load_dataset('AmazonScience/mintaka', name='en')
ds

Reusing dataset mintaka (/root/.cache/huggingface/datasets/AmazonScience___mintaka/en/1.0.0/bb35d95f07aed78fa590601245009c5f585efe909dbd4a8f2a4025ccf65bb11d)
100%|██████████| 3/3 [00:00<00:00, 267.76it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'lang', 'question', 'answerText', 'category', 'complexityType', 'questionEntity', 'answerEntity'],
        num_rows: 14000
    })
    validation: Dataset({
        features: ['id', 'lang', 'question', 'answerText', 'category', 'complexityType', 'questionEntity', 'answerEntity'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['id', 'lang', 'question', 'answerText', 'category', 'complexityType', 'questionEntity', 'answerEntity'],
        num_rows: 4000
    })
})

In [5]:
results_validation_df: pd.DataFrame = pd.read_csv('./mintaka_results_validation.csv') # validation

with open('to_subgraphs/mintaka_validation.jsonl', 'w') as f:
    for data_line in prepare_data(ds['validation'].to_pandas(), results_validation_df):
        f.write(ujson.dumps(data_line)+'\n')

100%|██████████| 2000/2000 [00:05<00:00, 365.25it/s]


In [6]:
results_train_df: pd.DataFrame = pd.read_csv('./mintaka_results_train.csv') # train

with open('to_subgraphs/mintaka_train.jsonl', 'w') as f:
    for data_line in prepare_data(ds['train'].to_pandas(), results_train_df):
        f.write(ujson.dumps(data_line)+'\n')

100%|██████████| 14000/14000 [00:13<00:00, 1021.77it/s]


In [7]:
results_test_df: pd.DataFrame = pd.read_csv('./mintaka_results_test.csv') # test

with open('to_subgraphs/mintaka_test.jsonl', 'w') as f:
    for data_line in prepare_data(ds['test'].to_pandas(), results_test_df):
        f.write(ujson.dumps(data_line)+'\n')

100%|██████████| 4000/4000 [00:03<00:00, 1331.76it/s]
